# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.6 MB/s eta 0:00:00


In [ ]:
import optuna

optuna.__version__

'4.5.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [ ]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection


def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)  # Define the model.

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()  # Train and evaluate the model.


print("Accuracy: {}".format(objective()))

Accuracy: 0.9533333333333333


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [ ]:
import optuna


def objective(trial):
    iris = sklearn.datasets.load_iris()

    n_estimators = trial.suggest_int("n_estimators", 2, 20)
    max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2025-09-20 21:37:19,577] A new study created in memory with name: no-name-77e4a08b-d69d-47f9-9252-cd81289f6c53
[I 2025-09-20 21:37:19,618] Trial 0 finished with value: 0.96 and parameters: {'n_estimators': 3, 'max_depth': 5.026098324880885}. Best is trial 0 with value: 0.96.
[I 2025-09-20 21:37:19,676] Trial 1 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 10, 'max_depth': 5.6683063850390765}. Best is trial 1 with value: 0.9666666666666667.
[I 2025-09-20 21:37:19,745] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 12, 'max_depth': 2.8563433387162407}. Best is trial 1 with value: 0.9666666666666667.
[I 2025-09-20 21:37:19,773] Trial 3 finished with value: 0.94 and parameters: {'n_estimators': 2, 'max_depth': 3.0391311052501147}. Best is trial 1 with value: 0.9666666666666667.
[I 2025-09-20 21:37:19,812] Trial 4 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 3, 'max_depth': 23.55191813673128}. Best i

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 13, 'max_depth': 3.941609657609293}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [ ]:
import sklearn.svm


def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical("classifier", ["RandomForest", "SVC"])

    if classifier == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 2, 20)
        max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth
        )
    else:
        c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)

        clf = sklearn.svm.SVC(C=c, gamma="auto")

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2025-09-20 21:38:17,197] A new study created in memory with name: no-name-244f99e1-6bbe-4ab9-846e-6d2f52a26ee6
[I 2025-09-20 21:38:17,219] Trial 0 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 40303372.448844746}. Best is trial 0 with value: 0.96.
[I 2025-09-20 21:38:17,249] Trial 1 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 1.0639991511658963e-05}. Best is trial 0 with value: 0.96.
[I 2025-09-20 21:38:17,329] Trial 2 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 13, 'max_depth': 23.7109626605792}. Best is trial 0 with value: 0.96.
[I 2025-09-20 21:38:17,369] Trial 3 finished with value: 0.7533333333333334 and parameters: {'classifier': 'RandomForest', 'n_estimators': 3, 'max_depth': 1.9708018436651056}. Best is trial 0 with value: 0.96.
[I 2025-09-20 21:38:17,388] Trial 4 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 14071.808659724187}. Best is trial 0 wit

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 4.009755473381441}


### Plotting the study

Plotting the optimization history of the study.

In [ ]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [ ]:
optuna.visualization.plot_slice(study)

A slice plot in Optuna is a visualization that helps you understand how individual hyperparameters affect the objective value (e.g., accuracy, loss, AUC) across all the trials.

Here’s the breakdown:

Meaning of Slice Plot

* Each subplot corresponds to one hyperparameter.

* On the x-axis → values of that hyperparameter tried in different trials.

* On the y-axis → the corresponding objective values (the metric you are optimizing).

Each dot = one trial’s result.

It helps spot:

* Trends → e.g., higher learning rate tends to give worse loss.

* Ranges that work well → clusters of good performance.

* Interactions (when viewed with other plots like contour plots).

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

A contour plot in Optuna is a visualization that shows how two hyperparameters interact and how their combinations affect the objective value.

**Meaning of Contour Plot**

* X-axis & Y-axis → two selected hyperparameters.

* Colors (contours) → represent the objective value (e.g., accuracy, loss).

* Darker/warmer colors = better (depending on whether you minimize or maximize).

* Each trial is also shown as a point on the plot.

**What It Tells You**

* Best regions of hyperparameter space → where the optimal values cluster.
* Interactions → whether two parameters influence each other.
Example: A learning rate of 0.01 might only work well if batch size is small.

Search efficiency → shows which areas of the parameter space have been explored and which remain sparse.

**Example**

If tuning: Learning rate (x-axis) and Batch size (y-axis):

* The contour plot might show a “sweet spot” region (e.g., lr=0.001–0.005 and batch size=64–128) where performance is highest.

* In short: a contour plot reveals the relationship between two hyperparameters and the regions where they jointly give good results.

In [ ]:
optuna.visualization.plot_contour(study, params=["n_estimators", "max_depth"])